In [1]:
import os
import sys
from datetime import datetime 
from datetime import timedelta
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook

In [5]:
# Set-up directory

disk_name = 'FRED SHONE' # Memory Stick
disk_location = '/Volumes'
disk_path = os.path.join(disk_location, disk_name)
print(disk_path)

/Volumes/FRED SHONE


In [6]:
# Access daily sub-directories of images

folders = sorted(os.listdir(disk_path)) # Assumes sorting by name is ok - but this won't work for changes in year

# Filter for days by trying to convert to datetime

day_directories = []
for folder in folders:
    try: 
        datetime.strptime(folder, '%m-%d-%Y')
        day_directories.append(folder)
    except: pass
    
print(day_directories)

['01-03-2015', '01-04-2015']


In [7]:
# Check for number of images in day directories

for day_directory in day_directories:
    day_dir_path = os.path.join(disk_path, day_directory)
    image_paths = os.listdir(day_dir_path)
    image_paths = sorted([image for image in os.listdir(day_dir_path) if not image.startswith('.')])
    image_count = len(image_paths)
    print('{}: {} images'.format(day_directory, image_count))   
    #Wilkins building listed as opening hours from 8am to 11pm = 15 hours
    frame_rate = image_count/(15*60*60)
    print('frame rate assumed to be ~{} frames per second'.format(frame_rate))

01-03-2015: 16684 images
frame rate assumed to be ~0.30896296296296294 frames per second
01-04-2015: 16680 images
frame rate assumed to be ~0.3088888888888889 frames per second


In [8]:
#
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

/Users/fredshone/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
# This is needed to display the images.
%matplotlib inline
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append('object_detection')

In [10]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [11]:
# What model to download.
MODEL_NAME = 'faster_rcnn_inception_v2_coco_2017_11_08'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('object_detection', 'data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [12]:
print(os.getcwd())

/Users/fredshone/Projects/Dissertation


In [13]:
# Download Model
opener = urllib.request.opener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

KeyboardInterrupt: 

In [14]:
#Load Frozen model into memory
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [15]:
# Load label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [16]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [17]:
# Set-up motion detector
trigger = 0.0001 # Total pixel difference to trigger motion detection
warning_trigger = 10000 # Total pixel difference to trigger verbose image plot

verbose = False 
start_time = '08:00' # Assumes camera turned on at 8am every day
frame_step = 3 # Seconds

detect_objects = True
detection_threshold = 0.5

In [153]:
trigger_record = [] # Record of triggers

In [154]:
for day_directory in tqdm_notebook(day_directories, desc='Days:'):
    '''
    print('------------------------------')
    print('>>> Processing images for {}'.format(day_directory))
    '''
    day_dir_path = os.path.join(disk_path, day_directory)
    images = sorted([image for image in os.listdir(day_dir_path) if not image.startswith('.')])
    
    date = datetime.strptime(day_directory,'%m-%d-%Y')
    start_datetime = datetime.strptime(day_directory+' '+start_time,'%m-%d-%Y %H:%M')
    
    background = None # Initialise with no background
    
    num_images = len(images) # Get amount of images for progress bar
        
    for image in tqdm_notebook(images, desc='{} progress:'.format(date)):
        
        image_path = os.path.join(day_dir_path, image)
        img = cv2.imread(image_path)
        
        # Reduce image size to 1%
        thumb = cv2.resize(img, None, fx=.1, fy=.1, interpolation = cv2.INTER_AREA)
        thumb = cv2.cvtColor(thumb, cv2.COLOR_BGR2GRAY)

        if background is None: # Initialise background for first pass
            background = thumb
            
        delta = cv2.absdiff(background, thumb)
        thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
        pixel_diff = cv2.sumElems(thresh)[0] # Need to check that this is the absolute difference
        
        if pixel_diff > img.size*trigger:
            
            # Extract frame number from imaga path
            image_index = [int(s) for s in list(image) if s.isdigit()]
            image_index = ''.join(str(x) for x in image_index) 
            image_index = int(image_index)
            
            # Calculate time stamp of frame
            time_delta = timedelta(seconds=(frame_step*image_index))
            time_stamp = start_datetime + time_delta
            
            if verbose:
                '''
                print('---------------')
                print(image) 
                print('Movement detected at {} with pixel delta of {}:'.format(str(time_stamp), pixel_diff))
                '''
                
                if pixel_diff < warning_trigger:
                    f, axarr = plt.subplots(1,3, figsize=(15,5))
                    axarr[0].imshow(background)
                    axarr[1].imshow(thresh)
                    axarr[2].imshow(thumb)
                    plt.show()

            # Object Detection
            #PATH_TO_TEST_IMAGES_DIR = 'test_images'
            #TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}.jpg'.format(i)) for i in range(1, 20) ]

            # Size, in inches, of the output images.
            IMAGE_SIZE = (18,12)
            
            with detection_graph.as_default():
              with tf.Session(graph=detection_graph) as sess:
                
                # Definite input and output Tensors for detection_graph
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Each box represents a part of the image where a particular object was detected.
                detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Each score represent how level of confidence for each of the objects.
                # Score is shown on the result image, together with the class label.
                detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
                detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                
                image = Image.open(image_path)
                #print('Image {} Loaded'.format(image_path))
                
                # the array based representation of the image will be used later in order to prepare the
                # result image with boxes and labels on it.
                #image_np = load_image_into_numpy_array(image)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(img, axis=0)
                # Actual detection.
                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores, detection_classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                # Select observations scored above threshold (default 0.5)
                boxes = boxes[scores>detection_threshold]
                classes = classes[scores>detection_threshold]
                scores = scores[scores>detection_threshold]
                num = len(classes)
                if num:
                    record = (time_stamp, (boxes, scores, classes, num))
                    trigger_record.append(record) # Add to record
                    print('{} detections recorded at {}'.format(num, str(time_stamp)))
                    for c, s in zip(classes, scores):
                        print('{}: {}%'.format(category_index[c]['name'], s))
                # Visualization of the results of a detection.
                '''
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)
                    '''
                #plt.figure(figsize=IMAGE_SIZE)
                #plt.imshow(image_np)
                #SAVE_IMAGE_PATH = os.path.join(PATH_TO_TEST_IMAGES_DIR, 'bbox{}.jpg'.format(index))
                #plt.savefig(SAVE_IMAGE_PATH)
                #print(SAVE_IMAGE_PATH + ' saved.')
            
            if len(trigger_record) > 20:
                break
            
        #i = int(100*index/(num_images-1))
        '''
        sys.stdout.write('\r')
        sys.stdout.write('Movement detected at {} with pixel delta of {}:'.format(str(time_stamp), pixel_diff))
        sys.stdout.write("[%-100s] %d%%" % ('='*i, i))
        sys.stdout.flush()
        '''
        background = thumb # Set background for next frame        

HBox(children=(IntProgress(value=0, description='Days:', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='2015-01-03 00:00:00 progress:', max=16684), HTML(value='')))

1 detections recorded at 2015-01-03 08:00:45
person: 0.9367648363113403%
3 detections recorded at 2015-01-03 08:01:36
person: 0.99587482213974%
person: 0.9759001135826111%
person: 0.912125825881958%
4 detections recorded at 2015-01-03 08:01:39
person: 0.999408483505249%
person: 0.9803464412689209%
person: 0.6700149774551392%
chair: 0.5471654534339905%
1 detections recorded at 2015-01-03 08:02:45
person: 0.978578507900238%
1 detections recorded at 2015-01-03 08:02:48
person: 0.9220980405807495%
1 detections recorded at 2015-01-03 08:11:18
person: 0.9960786700248718%
1 detections recorded at 2015-01-03 08:11:21
person: 0.9973539113998413%
1 detections recorded at 2015-01-03 08:11:24
person: 0.9088805913925171%
1 detections recorded at 2015-01-03 08:11:33
person: 0.9607324004173279%
1 detections recorded at 2015-01-03 08:11:36
person: 0.9613279104232788%
1 detections recorded at 2015-01-03 08:11:39
person: 0.9811680316925049%
1 detections recorded at 2015-01-03 08:11:42
person: 0.98070567

HBox(children=(IntProgress(value=0, description='2015-01-04 00:00:00 progress:', max=16680), HTML(value='')))

1 detections recorded at 2015-01-04 08:01:06
chair: 0.8469684720039368%


In [149]:
for r in trigger_record:
    for c, s in zip(r[1][2], r[1][1]):
        print('{}: {}% at {}'.format(category_index[c]['name'], s, r[0]))

person: 0.9611889123916626% at 2015-01-03 08:00:45
person: 0.9961543679237366% at 2015-01-03 08:01:36
person: 0.9777339100837708% at 2015-01-03 08:01:36
person: 0.9081301689147949% at 2015-01-03 08:01:36
person: 0.7179996967315674% at 2015-01-03 08:01:36
person: 0.9993106126785278% at 2015-01-03 08:01:39
person: 0.9560275077819824% at 2015-01-03 08:01:39
person: 0.7579991817474365% at 2015-01-03 08:01:39
person: 0.5621638298034668% at 2015-01-03 08:01:39
person: 0.9841618537902832% at 2015-01-03 08:02:45
person: 0.9181041121482849% at 2015-01-03 08:02:48
person: 0.9979661703109741% at 2015-01-03 08:11:18
person: 0.9977293610572815% at 2015-01-03 08:11:21
person: 0.8208078145980835% at 2015-01-03 08:11:24
person: 0.9628744125366211% at 2015-01-03 08:11:33
person: 0.9804023504257202% at 2015-01-03 08:11:36
person: 0.9813427329063416% at 2015-01-03 08:11:39
person: 0.9770221710205078% at 2015-01-03 08:11:42
person: 0.86475670337677% at 2015-01-03 08:11:45
person: 0.999150276184082% at 201

In [ ]:
import pandas as pd

In [ ]:
motion_detections = pd.DataFrame({'timestamp':trigger_record})

In [ ]:
start_time = min(motion_detections.timestamp).replace(hour=0, minute=0, second=0).to_pydatetime()
end_time = max(motion_detections.timestamp).replace(hour=0, minute=0, second=0).to_pydatetime() + timedelta(days=1)

In [ ]:
time_range = pd.date_range(start=start_time, end=end_time, freq='H')
time_range_counter = pd.DataFrame({'timestamps':all_range, 'counter':0}).set_index('timestamps')

In [ ]:
for detection in motion_detections.timestamp:
    detection_time = detection.replace(minute=0, second=0)
    time_range_counter.counter[detection_time] += 1

In [ ]:
my_colors = 'b'
time_range_counter.plot(kind='bar',
             color=my_colors,
             alpha=0.5,
             figsize=(16,8),
             title='Motion Detections')


In [ ]:
hour_motion_detections = motion_detections.groupby(motion_detections.timestamp.dt.hour).count()

In [ ]:
my_colors = 'r'
hour_motion_detections.plot(kind='bar',
             color=my_colors,
             alpha=0.5,
             figsize=(16,8),
             title='Av. Motion Detections')

In [ ]:
boxes

In [ ]:
boxes.shape

In [ ]:
scores.shape

In [ ]:
classes.shape

In [ ]:
num.shape

In [ ]:
classes

In [ ]:
category_index[88.]['name']

In [ ]:
len(category_index)

In [ ]:
for i in range(1,len(category_index)+1):
    try: 
        print(category_index[i]['name'])
    except:
        pass

In [ ]:
category_index